In [50]:
import numpy as np
import cv2
import face_recognition
import imutils
import pickle


In [51]:
videofile = 'team-rotated.mp4'
path_prefix = 'C:/Users/jshru_000/bootcamp1/FINAL/faces/'
pickle_path = path_prefix + 'faces.pkl'


In [52]:
def resizeframe(image, width=500):
    w = float(width)
    r = w / image.shape[1]
    dim = (width, int(image.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    xr = dim[0]/float(image.shape[1])
    yr = dim[1]/float(image.shape[0])
    return resized


In [53]:
def draw_str(dst, x, y, s):
    cv2.putText(dst, s, (x+1, y+1), cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 0, 0), thickness =2, lineType=cv2.LINE_AA)
    cv2.putText(dst, s, (x, y), cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 0, 255), lineType=cv2.LINE_AA)


In [54]:
def savefacesfromframes(frame):
    facelist = []
     # facedata = {fname1:encodings1, fname2:encodings2,...}
    metadata = get_faces(frame)
    if not metadata == {}:
         with open(pickle_path,'ab') as f:
            pickle.dump(metadata, f)
    return(len(metadata))

In [55]:
def get_faces(frame):
    model = 'hog' # 'cnn'
    metadata = {}
    global face_counter
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])
    boxes = face_recognition.face_locations(rgb,model=model)
    if not boxes == []:
        if not boxes == []:
            encodings = face_recognition.face_encodings(rgb, boxes)
            assert len(encodings) == len(boxes)
            for i, (top, right, bottom, left) in enumerate(boxes):
                top = int(top * r)
                right = int(right * r)
                bottom = int(bottom * r)
                left = int(left * r)
                thisFace = frame[top:bottom,left:right]
                thisFace = cv2.resize(thisFace, (96, 96))
                fname = 'face_'+str(face_counter)+'_'+str(i)+'.jpg'
                cv2.imwrite(path_prefix+fname, thisFace)
                metadata[fname] = encodings[i]
            face_counter += 1
    return(metadata)

In [56]:
cap1 = cv2.VideoCapture(videofile)
w1 = int(cap1.get(3))
h1 = int(cap1.get(4))
face_counter = 0
flag = True
frameno = 0
frameinterval = 1 #10
while cap1.isOpened():
    ret1, frame1 = cap1.read()
    frameno += 1
    if not ((frameno % frameinterval) == 0):
        continue
    if ret1 :
        frame1 = resizeframe(frame1)
        frame = frame1.copy()
        cv2.imshow('Frames- pause p; resume r; quit q',frame)
        if cv2.waitKey(1) & 0xFF == ord('p'): #pause
            while (True):
                if flag:
                    nooffaces = savefacesfromframes(frame)
                    draw_str(frame, 20, 20, 'No of faces identified: '+ str(nooffaces))
                    cv2.imshow('Frames- pause p; resume r; quit q',frame)
                    flag = False
                if cv2.waitKey(1) & 0xFF == ord('r'): #resume
                    flag = True
                    break
        if cv2.waitKey(10) & 0xFF == ord('q'): #quit
            break
    else:
         break
cap1.release()
cv2.destroyAllWindows()


In [57]:
data = []
with open(pickle_path, 'rb') as f:
    try:
        while True:
            data.append(pickle.load(f))
    except EOFError:
        pass


In [58]:
fdata = {}
for thisdata in data:
    fdata.update(thisdata)

In [59]:
len(fdata), type(fdata)

(125, dict)